In [ ]:
import asyncio
from typing import List, Dict, Optional, Any, Set
from enum import Enum
from pydantic import BaseModel, Field
from datetime import datetime
from collections import deque
import json
import os
from pathlib import Path

# Import the base agent framework (assumed to be implemented)
from base_agent import BaseAgent, MessageType, Message

# Define programmer expertise areas
class ExpertiseArea(str, Enum):
    PYTHON = "python"
    JAVASCRIPT = "javascript"
    DOCKER = "docker"
    KUBERNETES = "kubernetes"
    DATABASE = "database"
    NETWORKING = "networking"
    LINUX = "linux"
    SECURITY = "security"
    CLOUD = "cloud"

# Define task models
class Task(BaseModel):
    id: str
    title: str
    description: str
    required_expertise: List[ExpertiseArea]
    priority: int = 1
    deadline: Optional[datetime] = None
    assigned_at: Optional[datetime] = None
    completed_at: Optional[datetime] = None
    status: str = "pending"  # pending, in_progress, completed, failed

# Command execution record for learning
class CommandExecutionRecord(BaseModel):
    command: str
    stdout: str
    stderr: str
    syslog: Optional[str] = None
    exit_code: int
    system_state_before: Dict[str, Any]
    system_state_after: Dict[str, Any]
    execution_time: float
    timestamp: datetime = Field(default_factory=datetime.now)
    interaction_history: List[Dict[str, str]] = []  # For interactive commands
    agent_thoughts: List[Dict[str, Any]] = []

# Thought types for internal processing
class ThoughtType(str, Enum):
    OBSERVATION = "observation"
    ANALYSIS = "analysis"
    DECISION = "decision"
    LEARNING = "learning"
    REFLECTION = "reflection"
    PLANNING = "planning"

# Thought structure
class Thought(BaseModel):
    type: ThoughtType
    content: str
    confidence: float = 1.0
    related_command: Optional[str] = None
    related_output: Optional[str] = None
    timestamp: datetime = Field(default_factory=datetime.now)
    tags: List[str] = []
    
# Programmer metrics
class ProgrammerMetrics(BaseModel):
    tasks_completed: int = 0
    tasks_failed: int = 0
    commands_executed: int = 0
    command_success_rate: float = 1.0
    average_task_completion_time: float = 0.0
    learning_activities: List[str] = []
    skill_improvements: Dict[str, float] = {}

class ProgrammerAgent(BaseAgent):
    def __init__(
        self,
        name: str,
        model_path: str,
        expertise: List[ExpertiseArea],
        knowledge_base_paths: List[str],
        max_memory_size: int = 1000,
        max_context_size: int = 2048
    ):
        super().__init__(name=name, agent_type="programmer")
        self.model_path = model_path
        self.expertise = expertise
        self.knowledge_base_paths = knowledge_base_paths
        self.current_tasks: List[Task] = []
        self.completed_tasks: List[Task] = []
        self.test_results: List[Dict[str, Any]] = []
        self.command_history: List[CommandExecutionRecord] = []
        self.metrics = ProgrammerMetrics()
        
        # Internal thought process
        self.thoughts = deque(maxlen=max_memory_size)
        self.thought_patterns = {}  # Patterns discovered in thoughts
        self.thought_clusters = {}  # Clusters of related thoughts
        
        # Context management
        self.max_context_size = max_context_size
        self.current_context = []
        
        # LLM initialization will be handled in initialize()
        self.llm = None
        
    async def initialize(self):
        """Initialize the agent, load the LLM, and prepare knowledge base"""
        # LLM initialization will be implemented with the LLM deployment part
        # For now, we'll use a placeholder
        self.llm = await self._load_model(self.model_path)
        
        # Load knowledge from manuals
        self.knowledge_base = await self._load_knowledge_base()
        
        print(f"Programmer Agent {self.name} initialized with expertise: {', '.join([e.value for e in self.expertise])}")
    
    async def _load_model(self, model_path: str):
        """Load the small LLM model - implementation will vary based on framework choice"""
        # Placeholder for actual model loading
        # This will be replaced with actual implementation in the LLM deployment section
        return {"name": "placeholder_model", "path": model_path}
    
    async def _load_knowledge_base(self) -> Dict[str, Any]:
        """Load knowledge from PDFs or web pages"""
        # Placeholder for knowledge base loading
        knowledge = {}
        for path in self.knowledge_base_paths:
            if path.endswith('.pdf'):
                # Load PDF knowledge
                knowledge[os.path.basename(path)] = {"type": "pdf", "content": "PDF content placeholder"}
            elif path.startswith('http'):
                # Load web page knowledge
                knowledge[path] = {"type": "webpage", "content": "Web content placeholder"}
        return knowledge
    
    async def process_message(self, message: Message) -> Optional[Message]:
        """Process incoming messages from other agents"""
        # Generate thoughts about the message
        thought = Thought(
            type=ThoughtType.OBSERVATION,
            content=f"Received message of type {message.type} from {message.sender}",
            related_command=message.content.get("command") if isinstance(message.content, dict) else None
        )
        self.add_thought(thought)
        
        # Process different message types
        if message.type == MessageType.TASK_ASSIGNMENT:
            return await self._handle_task_assignment(message)
        elif message.type == MessageType.COMMAND_REQUEST:
            return await self._handle_command_request(message)
        elif message.type == MessageType.FEEDBACK:
            return await self._handle_feedback(message)
        
        # Default acknowledgment
        return Message(
            sender=self.name,
            recipient=message.sender,
            type=MessageType.ACKNOWLEDGMENT,
            content={"status": "received", "message_id": message.id}
        )
    
    async def _handle_task_assignment(self, message: Message) -> Message:
        """Handle task assignment from project manager"""
        task_data = message.content.get("task", {})
        task = Task(**task_data)
        
        # Check if we have the expertise for this task
        has_expertise = any(exp in self.expertise for exp in task.required_expertise)
        
        if has_expertise:
            # Accept the task
            task.status = "in_progress"
            task.assigned_at = datetime.now()
            self.current_tasks.append(task)
            
            # Generate planning thought
            planning_thought = Thought(
                type=ThoughtType.PLANNING,
                content=f"Planning approach for task: {task.title}",
                tags=[exp.value for exp in task.required_expertise]
            )
            self.add_thought(planning_thought)
            
            return Message(
                sender=self.name,
                recipient=message.sender,
                type=MessageType.TASK_ACCEPTED,
                content={"task_id": task.id, "status": "in_progress"}
            )
        else:
            # Reject the task due to lack of expertise
            return Message(
                sender=self.name,
                recipient=message.sender,
                type=MessageType.TASK_REJECTED,
                content={
                    "task_id": task.id, 
                    "reason": "Lack of required expertise",
                    "available_expertise": [exp.value for exp in self.expertise]
                }
            )
    
    async def _handle_command_request(self, message: Message) -> Message:
        """Handle request to execute a command"""
        command = message.content.get("command", "")
        server = message.content.get("server", "")
        
        # Generate analysis thought
        analysis_thought = Thought(
            type=ThoughtType.ANALYSIS,
            content=f"Analyzing command: {command} for server {server}",
            related_command=command,
            confidence=0.9
        )
        self.add_thought(analysis_thought)
        
        # For now, we'll just acknowledge the command
        # In a real implementation, this would execute the command and process results
        return Message(
            sender=self.name,
            recipient=message.sender,
            type=MessageType.COMMAND_RESPONSE,
            content={"command": command, "status": "acknowledged"}
        )
    
    async def _handle_feedback(self, message: Message) -> Message:
        """Handle feedback from project manager or other agents"""
        feedback = message.content.get("feedback", "")
        task_id = message.content.get("task_id", None)
        
        # Generate learning thought
        learning_thought = Thought(
            type=ThoughtType.LEARNING,
            content=f"Learning from feedback: {feedback}",
            confidence=0.85,
            tags=["feedback", f"task_{task_id}"] if task_id else ["feedback"]
        )
        self.add_thought(learning_thought)
        
        # Update metrics based on feedback
        if "positive" in feedback.lower():
            self.metrics.skill_improvements["feedback_response"] = self.metrics.skill_improvements.get("feedback_response", 0) + 0.1
        
        return Message(
            sender=self.name,
            recipient=message.sender,
            type=MessageType.ACKNOWLEDGMENT,
            content={"status": "feedback_received", "message": "Thank you for the feedback"}
        )
    
    def add_thought(self, thought: Thought):
        """Add a thought to the agent's thought process"""
        self.thoughts.append(thought)
        
        # Process the thought to identify patterns
        self._process_thought(thought)
    
    def _process_thought(self, thought: Thought):
        """Process a thought to identify patterns and update internal knowledge"""
        # Extract keywords from thought
        keywords = set(thought.content.lower().split())
        
        # Update thought patterns
        for keyword in keywords:
            if keyword not in self.thought_patterns:
                self.thought_patterns[keyword] = []
            self.thought_patterns[keyword].append(thought)
        
        # Cluster related thoughts (simplified version)
        for tag in thought.tags:
            if tag not in self.thought_clusters:
                self.thought_clusters[tag] = []
            self.thought_clusters[tag].append(thought)
    
    async def execute_command(self, command: str, server: str) -> CommandExecutionRecord:
        """Execute a command on a server and record the results"""
        # In a real implementation, this would connect to the server and execute the command
        # For now, we'll simulate command execution
        
        # Generate decision thought
        decision_thought = Thought(
            type=ThoughtType.DECISION,
            content=f"Decided to execute command: {command} on server {server}",
            related_command=command,
            confidence=0.9
        )
        self.add_thought(decision_thought)
        
        # Simulate command execution
        stdout = f"Simulated output for {command}"
        stderr = ""
        exit_code = 0
        execution_time = 0.5
        
        # Create execution record
        record = CommandExecutionRecord(
            command=command,
            stdout=stdout,
            stderr=stderr,
            exit_code=exit_code,
            system_state_before={"status": "before"},
            system_state_after={"status": "after"},
            execution_time=execution_time,
            agent_thoughts=[
                decision_thought.dict(),
                Thought(
                    type=ThoughtType.OBSERVATION,
                    content=f"Observed command output: {stdout}",
                    related_command=command,
                    related_output=stdout,
                    confidence=1.0
                ).dict()
            ]
        )
        
        # Update metrics
        self.metrics.commands_executed += 1
        self.command_history.append(record)
        
        return record
    
    async def complete_task(self, task_id: str, status: str = "completed", results: Dict[str, Any] = None):
        """Mark a task as completed and report results"""
        for i, task in enumerate(self.current_tasks):
            if task.id == task_id:
                task.status = status
                task.completed_at = datetime.now()
                
                # Generate reflection thought
                reflection_thought = Thought(
                    type=ThoughtType.REFLECTION,
                    content=f"Reflected on completed task {task.title} with status {status}",
                    confidence=0.9,
                    tags=[f"task_{task_id}", status]
                )
                self.add_thought(reflection_thought)
                
                # Update metrics
                if status == "completed":
                    self.metrics.tasks_completed += 1
                else:
                    self.metrics.tasks_failed += 1
                
                # Calculate task completion time
                if task.assigned_at:
                    completion_time = (datetime.now() - task.assigned_at).total_seconds()
                    # Update average completion time
                    total_completed = self.metrics.tasks_completed + self.metrics.tasks_failed
                    self.metrics.average_task_completion_time = (
                        (self.metrics.average_task_completion_time * (total_completed - 1) + completion_time) 
                        / total_completed
                    )
                
                # Move to completed tasks
                self.completed_tasks.append(task)
                self.current_tasks.pop(i)
                
                # Notify project manager
                await self.send_message(
                    "project_manager",
                    MessageType.TASK_COMPLETED,
                    {
                        "task_id": task_id,
                        "status": status,
                        "results": results or {},
                        "completion_time": task.completed_at - task.assigned_at if task.assigned_at else None
                    }
                )
                return True
        
        return False
    
    async def run_tests(self):
        """Run tests to improve skills when not working on tasks"""
        if not self.current_tasks:
            # Generate planning thought for self-improvement
            planning_thought = Thought(
                type=ThoughtType.PLANNING,
                content="Planning self-improvement through testing",
                confidence=0.8,
                tags=["self_improvement", "testing"]
            )
            self.add_thought(planning_thought)
            
            # Simulate running tests
            test_result = {
                "coverage": 85.5,
                "passed": 42,
                "failed": 3,
                "learning_points": ["Point 1", "Point 2"]
            }
            
            self.test_results.append(test_result)
            
            # Update learning metrics
            self.metrics.learning_activities.append("Unit Testing")
            self.metrics.skill_improvements["testing"] = self.metrics.skill_improvements.get("testing", 0) + 0.1
            
            # Generate learning thought from test results
            learning_thought = Thought(
                type=ThoughtType.LEARNING,
                content=f"Learned from test results: {test_result['passed']} passed, {test_result['failed']} failed",
                confidence=0.9,
                tags=["testing", "self_improvement"]
            )
            self.add_thought(learning_thought)
            
            return test_result
        
        return None
    
    async def request_help(self, problem: Dict[str, Any], recipient: str = "architect"):
        """Request help from architect or other specialists"""
        # Generate analysis thought about the problem
        analysis_thought = Thought(
            type=ThoughtType.ANALYSIS,
            content=f"Analyzed problem requiring help: {problem.get('description', '')}",
            confidence=0.7,
            tags=["help_request"]
        )
        self.add_thought(analysis_thought)
        
        # Send help request
        return await self.send_message(
            recipient,
            MessageType.HELP_REQUEST,
            {
                "problem": problem,
                "context": {
                    "expertise": [e.value for e in self.expertise],
                    "current_task": self.current_tasks[0].dict() if self.current_tasks else None,
                    "relevant_thoughts": [
                        t.dict() for t in list(self.thoughts)[-5:] 
                        if any(tag in t.tags for tag in problem.get("tags", []))
                    ]
                }
            }
        )
    
    async def get_relevant_thoughts(self, query: str, max_thoughts: int = 5) -> List[Thought]:
        """Retrieve relevant thoughts based on a query"""
        # Simple keyword matching for now
        keywords = set(query.lower().split())
        relevant_thoughts = []
        
        # Find thoughts related to the keywords
        for keyword in keywords:
            if keyword in self.thought_patterns:
                relevant_thoughts.extend(self.thought_patterns[keyword])
        
        # Remove duplicates and sort by timestamp (newest first)
        unique_thoughts = {}
        for thought in relevant_thoughts:
            if thought not in unique_thoughts:
                unique_thoughts[thought] = thought
        
        sorted_thoughts = sorted(
            unique_thoughts.values(), 
            key=lambda t: t.timestamp, 
            reverse=True
        )
        
        return sorted_thoughts[:max_thoughts]
    
    async def create_context_for_llm(self, query: str, task: Optional[Task] = None) -> Dict[str, Any]:
        """Create a context for the LLM based on the current state and query"""
        # Get relevant thoughts
        relevant_thoughts = await self.get_relevant_thoughts(query)
        
        # Create a lightweight context
        context = {
            "query": query,
            "expertise": [e.value for e in self.expertise],
            "current_task": task.dict() if task else None,
            "relevant_thoughts": [t.dict() for t in relevant_thoughts],
            "knowledge_summary": self._summarize_knowledge_for_query(query)
        }
        
        return context
    
    def _summarize_knowledge_for_query(self, query: str) -> Dict[str, str]:
        """Summarize relevant knowledge base entries for a query"""
        # Simple placeholder implementation
        # In a real system, this would use the LLM to find and summarize relevant knowledge
        summaries = {}
        keywords = set(query.lower().split())
        
        for key, entry in self.knowledge_base.items():
            content = entry.get("content", "")
            # Very simple matching - would be more sophisticated in real implementation
            if any(keyword in content.lower() for keyword in keywords):
                summaries[key] = f"Summary of {key}"
        
        return summaries
    
    async def get_command_recommendation(self, task: Task) -> str:
        """Get a command recommendation for a task from the LLM"""
        # Create context for the LLM
        context = await self.create_context_for_llm(task.description, task)
        
        # In a real implementation, this would use the LLM to generate a command
        # For now, return a placeholder
        return f"echo 'This is a placeholder command for task {task.id}'"
    
    async def run(self):
        """Main agent loop"""
        await self.initialize()
        
        while True:
            # Process messages
            messages = await self.check_messages()
            for message in messages:
                await self.process_message(message)
            
            # Work on current tasks
            for task in self.current_tasks:
                # Generate thought about working on the task
                working_thought = Thought(
                    type=ThoughtType.OBSERVATION,
                    content=f"Working on task {task.id}: {task.title}",
                    confidence=1.0,
                    tags=[f"task_{task.id}"]
                )
                self.add_thought(working_thought)
                
                # Get command recommendation
                command = await self.get_command_recommendation(task)
                
                # Execute command
                record = await self.execute_command(command, "target_server")
                
                # Check if task is complete
                if record.exit_code == 0:
                    # Simple condition for now
                    # In a real implementation, this would be more sophisticated
                    if "success" in record.stdout.lower():
                        await self.complete_task(task.id, "completed", {"output": record.stdout})
                else:
                    # Command failed
                    # Request help if needed
                    await self.request_help({
                        "description": f"Command failed: {command}",
                        "error": record.stderr,
                        "tags": [f"task_{task.id}", "error"]
                    })
            
            # Run tests if no current tasks
            if not self.current_tasks:
                await self.run_tests()
            
            # Sleep to avoid busy waiting
            await asyncio.sleep(1)